In [2]:
from PIL import Image
import os
from versatile_diffusion import Reconstructor
import torch
import h5py
from tqdm import tqdm

In [3]:
reconstructor = Reconstructor(device='cuda:1', cache_dir='../cache', ddim_steps=20, clip_only=True)

Reconstructor: Loading model... fp16: True, clip_only: True

#######################
# Running in eps mode #
#######################

making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Load pth from ../cache/kl-f8.pth
Load autoencoderkl with total 83653863 parameters,72921.759 parameter sum.
Load optimus_bert_connector with total 109489920 parameters,18936.836 parameter sum.
Load optimus_gpt2_connector with total 132109824 parameters,19327.192 parameter sum.
Load pth from ../cache/optimus-vae.pth
Load optimus_vae_next with total 241599744 parameters,-344611.688 parameter sum.


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Load clip_image_context_encoder with total 427616513 parameters,64007.510 parameter sum.


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Load clip_text_context_encoder with total 427616513 parameters,64007.510 parameter sum.
Load openai_unet_2d_next with total 859520964 parameters,100311.692 parameter sum.
Load openai_unet_0d_next with total 1706797888 parameters,250131.434 parameter sum.
Load vd_v2_0 with total 3746805485 parameters,206768.216 parameter sum.


In [3]:
prompt_im = Image.open('/home/naxos2-raid25/kneel027/home/kneel027/IP-Adapter/assets/images/woman.png')
text_clip = reconstructor.embed_text("A cat")
image_clip = reconstructor.embed_image(prompt_im)
print(image_clip.shape, text_clip.shape)
# reconstruction = reconstructor.reconstruct(c_t=text_clip)
# reconstruction = reconstructor.reconstruct(c_i=image_clip, c_t=text_clip)
# reconstruction
# reconstruction2 = reconstructor.reconstruct(image=reconstruction, c_t=text_clip, strength=0.85)
# reconstruction2

torch.Size([1, 257, 768]) torch.Size([1, 77, 768])


/export/raid1/home/kneel027/mamba/envs/mindeye_imagery_vd/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [4]:
f = h5py.File(f'../dataset/coco_images_224_float16.hdf5', 'r')
images = f['images']
emb_batch_size = 50
clip_image = torch.zeros((len(images), 257 * 768)).to("cpu")
for i in tqdm(range(len(images) // emb_batch_size), desc="Encoding images..."):
    batch_images = images[i * emb_batch_size:i * emb_batch_size + emb_batch_size]
    batch_embeddings = reconstructor.embed_image(torch.from_numpy(batch_images)).reshape(emb_batch_size, -1).detach().to("cpu")

Encoding images...:  55%|█████▍    | 799/1460 [10:38<09:42,  1.14it/s]  